## Market Basket Analysis With FP-Growth


In [1]:
#import thư viện
import pandas as pd
import numpy as np

In [2]:
#Đọc dữ liêu từ file
df = pd.read_csv('./data/input.csv', encoding='latin-1', dtype={'BillNo': str})

In [3]:
# Tiền xử lý dữ liệu
df.shape

(522064, 7)

In [4]:
df.head(10)

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
9,536367,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 3   Date        522064 non-null  object 
 4   Price       522064 non-null  float64
 5   CustomerID  388023 non-null  float64
 6   Country     522064 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [6]:
df.sort_values('Quantity')

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
217658,556690,printing smudges/thrown away,-9600,6/14/2011 10:37,0.00,NaN,United Kingdom
217659,556691,printing smudges/thrown away,-9600,6/14/2011 10:37,0.00,NaN,United Kingdom
217657,556687,Printing smudges/thrown away,-9058,6/14/2011 10:36,0.00,NaN,United Kingdom
111940,546152,throw away,-5368,3/9/2011 17:25,0.00,NaN,United Kingdom
414734,573596,"Unsaleable, destroyed.",-4830,10/31/2011 15:17,0.00,NaN,United Kingdom
...,...,...,...,...,...,...,...
405138,573008,WORLD WAR 2 GLIDERS ASSTD DESIGNS,4800,10/27/2011 12:26,0.21,12901.0,United Kingdom
72509,542504,NaN,5568,1/28/2011 12:03,0.00,NaN,United Kingdom
483371,578841,ASSTD DESIGN 3D PAPER STICKERS,12540,11/25/2011 15:57,0.00,13256.0,United Kingdom
59999,541431,MEDIUM CERAMIC TOP STORAGE JAR,74215,1/18/2011 10:01,1.04,12346.0,United Kingdom


In [7]:
df.sort_values('Price')

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
288774,A563187,Adjust bad debt,1,8/12/2011 14:52,-11062.06,NaN,United Kingdom
288773,A563186,Adjust bad debt,1,8/12/2011 14:51,-11062.06,NaN,United Kingdom
343819,568091,NaN,-3,9/23/2011 14:41,0.00,NaN,United Kingdom
40271,539856,BLACK KITCHEN SCALES,3,12/22/2010 14:41,0.00,NaN,United Kingdom
233693,558340,KINGS CHOICE BISCUIT TIN,3,6/28/2011 14:01,0.00,NaN,United Kingdom
...,...,...,...,...,...,...,...
258372,560373,Manual,1,7/18/2011 12:30,4287.63,NaN,United Kingdom
286674,562955,DOTCOM POSTAGE,1,8/11/2011 10:14,4505.17,NaN,United Kingdom
167329,551697,POSTAGE,1,5/3/2011 13:46,8142.75,16029.0,United Kingdom
288772,A563185,Adjust bad debt,1,8/12/2011 14:50,11062.06,NaN,United Kingdom


In [8]:
df.isnull().sum()

BillNo             0
Itemname        1455
Quantity           0
Date               0
Price              0
CustomerID    134041
Country            0
dtype: int64

In [9]:
itemCount = df.Itemname.nunique()
items = df.Itemname.unique()

print('Item count: ' + str(itemCount))
print(items)

Item count: 4185
['WHITE HANGING HEART T-LIGHT HOLDER' 'WHITE METAL LANTERN'
 'CREAM CUPID HEARTS COAT HANGER' ... 'lost'
 'CREAM HANGING HEART T-LIGHT HOLDER' 'PAPER CRAFT , LITTLE BIRDIE']


In [10]:
new_df = df.dropna()
new_df.shape

(388023, 7)

In [13]:
new_df.loc[df['Price'] < 0].shape

(0, 7)

In [14]:
new_df.loc[df['Quantity'] < 0].shape

(0, 7)

In [13]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 388023 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      388023 non-null  object 
 1   Itemname    388023 non-null  object 
 2   Quantity    388023 non-null  int64  
 3   Date        388023 non-null  object 
 4   Price       388023 non-null  float64
 5   CustomerID  388023 non-null  float64
 6   Country     388023 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 23.7+ MB


In [17]:
transaction_list = []

for i in new_df['BillNo'].unique():
    tlist = list(set(new_df.loc[new_df['BillNo'] == i]['Itemname']))
    if len(tlist) > 0:
        transaction_list.append(tlist)

print(len(transaction_list))

18163


In [22]:
print(transaction_list[:10])

[['SET 7 BABUSHKA NESTING BOXES', 'RED WOOLLY HOTTIE WHITE HEART.', 'WHITE METAL LANTERN', 'GLASS STAR FROSTED T-LIGHT HOLDER', 'WHITE HANGING HEART T-LIGHT HOLDER', 'CREAM CUPID HEARTS COAT HANGER', 'KNITTED UNION FLAG HOT WATER BOTTLE'], ['HAND WARMER RED POLKA DOT', 'HAND WARMER UNION JACK'], ['HOME BUILDING BLOCK WORD', "POPPY'S PLAYHOUSE BEDROOM", 'BOX OF VINTAGE ALPHABET BLOCKS', 'RECIPE BOX WITH METAL HEART', 'DOORMAT NEW ENGLAND', 'FELTCRAFT PRINCESS CHARLOTTE DOLL', 'ASSORTED COLOUR BIRD ORNAMENT', 'BOX OF VINTAGE JIGSAW BLOCKS', 'LOVE BUILDING BLOCK WORD', 'IVORY KNITTED MUG COSY', "POPPY'S PLAYHOUSE KITCHEN", 'BOX OF 6 ASSORTED COLOUR TEASPOONS'], ['YELLOW COAT RACK PARIS FASHION', 'RED COAT RACK PARIS FASHION', 'JAM MAKING SET WITH JARS', 'BLUE COAT RACK PARIS FASHION'], ['BATH BUILDING BLOCK WORD'], ['ALARM CLOCK BAKELIKE RED', 'STARS GIFT TAPE', 'SET 2 TEA TOWELS I LOVE LONDON', 'POSTAGE', 'ROUND SNACK BOXES SET OF4 WOODLAND', 'SPACEBOY LUNCH BOX', 'MINI PAINT SET VINTAGE

In [23]:
# import thư viện khai phá luật kết hợp
import mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules,fpgrowth

In [24]:
te = TransactionEncoder()
te_ary = te.fit(transaction_list).transform(transaction_list)
df2 = pd.DataFrame(te_ary, columns=te.columns_)

In [25]:
df2.head(10)

,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [26]:
MIN_SUP = 0.02
MIN_CONF = 0.5
frequent_itemsets = fpgrowth(df2, min_support=MIN_SUP, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=MIN_CONF)

In [27]:
frequent_itemsets

,support,itemsets
0,0.105654,(WHITE HANGING HEART T-LIGHT HOLDER)
1,0.024280,(HAND WARMER UNION JACK)
2,0.074767,(ASSORTED COLOUR BIRD ORNAMENT)
3,0.037384,(HOME BUILDING BLOCK WORD)
4,0.030006,(LOVE BUILDING BLOCK WORD)
...,...,...
235,0.020701,"(LUNCH BAG SPACEBOY DESIGN, LUNCH BAG SUKI DES..."
236,0.022353,"(LUNCH BAG BLACK SKULL., LUNCH BAG SUKI DESIGN)"
237,0.021142,"(LUNCH BAG RED RETROSPOT, LUNCH BAG APPLE DESIGN)"
238,0.020922,"(PARTY BUNTING, SPOTTY BUNTING)"


In [21]:
rules.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
51,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.028960,0.035952,0.023785,0.821293,22.844013,0.022743,5.394565,0.984743
49,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.035952,0.040412,0.027859,0.774885,19.174712,0.026406,4.262660,0.983196
53,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.028960,0.040412,0.022408,0.773764,19.146976,0.021238,4.241541,0.976038
67,(GARDENERS KNEELING PAD CUP OF TEA),(GARDENERS KNEELING PAD KEEP CALM),0.034741,0.041238,0.025381,0.730586,17.716476,0.023949,3.558700,0.977515
48,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.040412,0.035952,0.027859,0.689373,19.174712,0.026406,3.103557,0.987765
...,...,...,...,...,...,...,...,...,...,...
8,(JUMBO BAG RED RETROSPOT),(LUNCH BAG RED RETROSPOT),0.086605,0.069702,0.023069,0.266370,3.821547,0.017032,1.268075,0.808331
42,(JUMBO BAG RED RETROSPOT),(JUMBO BAG STRAWBERRY),0.086605,0.035512,0.022408,0.258741,7.286074,0.019333,1.301149,0.944555
15,(JUMBO BAG RED RETROSPOT),(JUMBO SHOPPER VINTAGE RED PAISLEY),0.086605,0.043055,0.021582,0.249205,5.788129,0.017854,1.274577,0.905668
47,(REGENCY CAKESTAND 3 TIER),(ROSES REGENCY TEACUP AND SAUCER),0.089578,0.040412,0.021307,0.237861,5.885928,0.017687,1.259073,0.911778
